In [1]:
import sqlalchemy as db
import pandas as pd
import json
import os
from datetime import datetime, timezone

import copy

# Time execution
from time import time

## Functions

In [2]:
def format_playlist(playlist):
    # Change the format of important columns in the playlist
    # Also remove the track_list and return it to process it later
    
    # Store the tracks for later
    track_list = playlist.pop('tracks')
    
    # Convert modified_at column into Date object
    timestamp = playlist['modified_at']
    playlist['modified_at'] = datetime.fromtimestamp(timestamp, timezone.utc).date()
    
    # Convert collaborative column into boolean
    # I don't know what values they're using to represent true/false values.
    # The following code stops the creation of the database if we find a value
    # we don't recognize
    colab = str(playlist['collaborative'])
    if colab in true_values:
        playlist['collaborative'] = True
    elif colab in false_values:
        playlist['collaborative'] = False
    else:
        raise ValueError('The following value is not in the allowed true/false list:' +
                         str(playlist['collaborative']),
                         playlist['collaborative'])
    
    # Remove the pos entry from each track and return it in the same order
    pos_list = []
    for track in track_list:
        pos = track.pop('pos')
        pos_list.append(pos)
    
    return playlist, track_list, pos_list

# Modified from
# https://stackoverflow.com/questions/9792664/converting-a-list-to-a-set-changes-element-order
# to work with dicts
def unique_dict(sequence):
    sequence_tuple = [tuple(d.items()) for d in sequence]
    
    seen = set()
    return [dict(x) for x in sequence_tuple if not (x in seen or seen.add(x))]

def find_new_tracks(track_list, track_cache):
    # Remove duplicate tracks
    track_unique = unique_dict(track_list)
    
    # Check which tracks are cached already
    uris = [track['track_uri'] for track in track_unique]
    duplicate = [ (U in track_cache) for U in uris]
    
    # Retain only new tracks
    new_tracks = []
    tid = len(track_cache)
    for i in range(len(track_unique)):
        if not duplicate[i]:
            # Add a 'tid' entry to the track
            track_unique[i]['tid'] = tid
            
            # Update new_tracks and cache
            new_tracks.append(track_unique[i])
            track_cache[uris[i]] = tid
            tid += 1
    
    return new_tracks, track_cache

def create_pairings(track_list, pos_list, pid_list, track_cache):
    # Create pairings table
    pairing_list = []
    
    for i in range(len(track_list)):
        track = track_list[i]
        pid = pid_list[i]
        pos = pos_list[i]
        
        # tid is the index of the track in the cache
        tid = track_cache[track['track_uri']]
        
        # Create the playlist-track pairing
        pair = {
            'pid': pid,
            'tid': tid,
            'pos': pos
        }
        
        # Add the pairing to the list
        pairing_list.append(pair)
    
    return(pairing_list)

## Preview data

In [3]:
data_folder = "spotify_million_playlist_dataset/data"

# Load one json file and display the column names
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

json_file = json_files[0]
json_file_path = os.path.join(data_folder, json_file)
with open(json_file_path) as json_file:
    data = json.load(json_file)

playlist = data['playlists'][0]

print(' Keys for playlists')
print('--------------------')
for key in data['playlists'][0].keys():
    print('   ' + key)
print()

print(' Keys for tracks')
print('-----------------')
for key in data['playlists'][0]['tracks'][0].keys():
    print('   ' + key)

 Keys for playlists
--------------------
   name
   collaborative
   pid
   modified_at
   num_tracks
   num_albums
   num_followers
   tracks
   num_edits
   duration_ms
   num_artists

 Keys for tracks
-----------------
   pos
   artist_name
   track_uri
   artist_uri
   track_name
   album_uri
   duration_ms
   album_name


## Create databases and connection

In [4]:
data_folder = "spotify_million_playlist_dataset/data"

engine = db.create_engine("sqlite:///MPD_sql.db")
conn = engine.connect()
metadata = db.MetaData()
 
# Playlists table
tb_playlists = db.Table('playlists', metadata,
                        db.Column('name', db.String(255), nullable=False),
                        db.Column('pid', db.Integer(), nullable=False, primary_key=True),
                        db.Column('collaborative', db.Boolean()),
                        db.Column('modified_at', db.Date()),
                        db.Column('num_tracks', db.Integer()),
                        db.Column('num_albums', db.Integer()),
                        db.Column('num_followers', db.Integer()),
                        db.Column('num_edits', db.Integer()),
                        db.Column('duration_ms', db.Integer()),
                        db.Column('num_artists', db.Integer())
                       )

# Tracks table
tb_tracks = db.Table('tracks', metadata,
                     db.Column('tid', db.Integer(), nullable=False, primary_key=True),
                     db.Column('track_name', db.String(255), nullable=False),
                     db.Column('artist_name', db.String(255)),
                     db.Column('album_name', db.String(255)),
                     db.Column('duration_ms', db.Integer()),
                     db.Column('track_uri', db.String(255), nullable=False),
                     db.Column('artist_uri', db.String(255)),
                     db.Column('album_uri', db.String(255))
                    )

# Table with playlist-track pairing
tb_pairings = db.Table('pairings', metadata,
                       db.Column('pid', db.Integer(), nullable=False),
                       db.Column('tid', db.Integer(), nullable=False),
                       db.Column('pos', db.Integer(), nullable=False)
                      )

metadata.create_all(engine) # Create the tables

true_values = ['true', '1']
false_values = ['false', '0']

Populate database. We iterate over all json files and insert all the playlists in a single file at the same time.

In [5]:
time_0 = time()    # Record starting time

# List to record all the tracks that are already in the database
# This prevents adding duplicate entries
track_cache = {}

# Load .json file names from disk
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

# Limit to the first few files
# json_files = json_files[:1]

# Iterate through the selected JSON files
for json_file_name in json_files:
    iter_start = time()
    print('Processing:', json_file_name)
    
    # Load file
    json_file_path = os.path.join(data_folder, json_file_name)
    json_file = open(json_file_path)
    data = json.load(json_file)
    
    # Lists to store pairings
    new_track_pairings = []
    new_pos_list = []
    pid_list = []
    
    # Process the playlists
    for playlist in data["playlists"]:
        # Format the playlist info and obtain its tracks
        playlist, track_list, pos_list = format_playlist(playlist)
        
        # Save formated tracks and positions
        new_track_pairings = new_track_pairings + track_list
        new_pos_list = new_pos_list + pos_list
        pid_list = pid_list + [playlist['pid']]*len(track_list)
    
    # Remove duplicate tracks and update cache
    unseen_tracks, track_cache = find_new_tracks(new_track_pairings, track_cache)
    
    # Create the playlist-track pairings
    new_pairings = create_pairings(new_track_pairings, new_pos_list, pid_list, track_cache)
    
    # Insert playlists
    query = db.insert(tb_playlists)
    ResultProxy = conn.execute(query, data["playlists"])

    # Insert tracks
    query = db.insert(tb_tracks)
    ResultProxy = conn.execute(query, unseen_tracks)

    # Insert pairings
    query = db.insert(tb_pairings)
    ResultProxy = conn.execute(query, new_pairings)
    
    iter_end = time()
    print('Success!')
    print('%0.3f sec' % (iter_end-iter_start) )
    print()

# Commit changes to the database
# conn.commit()

current_time = time()
print('END')
print('Total time: %0.3f sec' % float(current_time-time_0) )
print('Total time: %0.3f min' % float((current_time-time_0)/60) )

Processing: mpd.slice.0-999.json
Success!
1.098 sec

Processing: mpd.slice.1000-1999.json
Success!
1.027 sec

Processing: mpd.slice.10000-10999.json
Success!
0.923 sec

Processing: mpd.slice.100000-100999.json
Success!
1.015 sec

Processing: mpd.slice.101000-101999.json
Success!
0.953 sec

Processing: mpd.slice.102000-102999.json
Success!
0.928 sec

Processing: mpd.slice.103000-103999.json
Success!
0.955 sec

Processing: mpd.slice.104000-104999.json
Success!
1.017 sec

Processing: mpd.slice.105000-105999.json
Success!
1.004 sec

Processing: mpd.slice.106000-106999.json
Success!
0.986 sec

Processing: mpd.slice.107000-107999.json
Success!
0.969 sec

Processing: mpd.slice.108000-108999.json
Success!
0.997 sec

Processing: mpd.slice.109000-109999.json
Success!
1.000 sec

Processing: mpd.slice.11000-11999.json
Success!
1.078 sec

Processing: mpd.slice.110000-110999.json
Success!
0.979 sec

Processing: mpd.slice.111000-111999.json
Success!
0.958 sec

Processing: mpd.slice.112000-112999.json

Success!
1.109 sec

Processing: mpd.slice.220000-220999.json
Success!
1.087 sec

Processing: mpd.slice.221000-221999.json
Success!
1.068 sec

Processing: mpd.slice.222000-222999.json
Success!
1.120 sec

Processing: mpd.slice.223000-223999.json
Success!
1.000 sec

Processing: mpd.slice.224000-224999.json
Success!
1.103 sec

Processing: mpd.slice.225000-225999.json
Success!
1.048 sec

Processing: mpd.slice.226000-226999.json
Success!
1.088 sec

Processing: mpd.slice.227000-227999.json
Success!
1.044 sec

Processing: mpd.slice.228000-228999.json
Success!
1.136 sec

Processing: mpd.slice.229000-229999.json
Success!
0.983 sec

Processing: mpd.slice.23000-23999.json
Success!
1.071 sec

Processing: mpd.slice.230000-230999.json
Success!
1.015 sec

Processing: mpd.slice.231000-231999.json
Success!
1.008 sec

Processing: mpd.slice.232000-232999.json
Success!
1.036 sec

Processing: mpd.slice.233000-233999.json
Success!
1.196 sec

Processing: mpd.slice.234000-234999.json
Success!
1.071 sec

Proces

Success!
0.940 sec

Processing: mpd.slice.342000-342999.json
Success!
1.108 sec

Processing: mpd.slice.343000-343999.json
Success!
1.049 sec

Processing: mpd.slice.344000-344999.json
Success!
0.965 sec

Processing: mpd.slice.345000-345999.json
Success!
1.033 sec

Processing: mpd.slice.346000-346999.json
Success!
1.064 sec

Processing: mpd.slice.347000-347999.json
Success!
1.082 sec

Processing: mpd.slice.348000-348999.json
Success!
1.094 sec

Processing: mpd.slice.349000-349999.json
Success!
1.061 sec

Processing: mpd.slice.35000-35999.json
Success!
1.023 sec

Processing: mpd.slice.350000-350999.json
Success!
1.102 sec

Processing: mpd.slice.351000-351999.json
Success!
1.015 sec

Processing: mpd.slice.352000-352999.json
Success!
1.032 sec

Processing: mpd.slice.353000-353999.json
Success!
0.998 sec

Processing: mpd.slice.354000-354999.json
Success!
1.068 sec

Processing: mpd.slice.355000-355999.json
Success!
0.989 sec

Processing: mpd.slice.356000-356999.json
Success!
1.073 sec

Proces

Success!
1.053 sec

Processing: mpd.slice.464000-464999.json
Success!
1.154 sec

Processing: mpd.slice.465000-465999.json
Success!
0.996 sec

Processing: mpd.slice.466000-466999.json
Success!
1.073 sec

Processing: mpd.slice.467000-467999.json
Success!
0.968 sec

Processing: mpd.slice.468000-468999.json
Success!
1.111 sec

Processing: mpd.slice.469000-469999.json
Success!
1.023 sec

Processing: mpd.slice.47000-47999.json
Success!
1.158 sec

Processing: mpd.slice.470000-470999.json
Success!
1.035 sec

Processing: mpd.slice.471000-471999.json
Success!
1.135 sec

Processing: mpd.slice.472000-472999.json
Success!
1.035 sec

Processing: mpd.slice.473000-473999.json
Success!
1.109 sec

Processing: mpd.slice.474000-474999.json
Success!
1.016 sec

Processing: mpd.slice.475000-475999.json
Success!
1.087 sec

Processing: mpd.slice.476000-476999.json
Success!
1.005 sec

Processing: mpd.slice.477000-477999.json
Success!
1.085 sec

Processing: mpd.slice.478000-478999.json
Success!
1.069 sec

Proces

Success!
1.079 sec

Processing: mpd.slice.586000-586999.json
Success!
1.066 sec

Processing: mpd.slice.587000-587999.json
Success!
1.032 sec

Processing: mpd.slice.588000-588999.json
Success!
1.048 sec

Processing: mpd.slice.589000-589999.json
Success!
1.019 sec

Processing: mpd.slice.59000-59999.json
Success!
1.104 sec

Processing: mpd.slice.590000-590999.json
Success!
1.104 sec

Processing: mpd.slice.591000-591999.json
Success!
1.116 sec

Processing: mpd.slice.592000-592999.json
Success!
1.166 sec

Processing: mpd.slice.593000-593999.json
Success!
1.084 sec

Processing: mpd.slice.594000-594999.json
Success!
1.074 sec

Processing: mpd.slice.595000-595999.json
Success!
1.111 sec

Processing: mpd.slice.596000-596999.json
Success!
0.989 sec

Processing: mpd.slice.597000-597999.json
Success!
1.083 sec

Processing: mpd.slice.598000-598999.json
Success!
1.016 sec

Processing: mpd.slice.599000-599999.json
Success!
1.051 sec

Processing: mpd.slice.6000-6999.json
Success!
1.029 sec

Processing

Success!
1.155 sec

Processing: mpd.slice.707000-707999.json
Success!
1.090 sec

Processing: mpd.slice.708000-708999.json
Success!
1.302 sec

Processing: mpd.slice.709000-709999.json
Success!
1.061 sec

Processing: mpd.slice.71000-71999.json
Success!
1.308 sec

Processing: mpd.slice.710000-710999.json
Success!
1.096 sec

Processing: mpd.slice.711000-711999.json
Success!
1.260 sec

Processing: mpd.slice.712000-712999.json
Success!
1.425 sec

Processing: mpd.slice.713000-713999.json
Success!
1.286 sec

Processing: mpd.slice.714000-714999.json
Success!
1.042 sec

Processing: mpd.slice.715000-715999.json
Success!
1.100 sec

Processing: mpd.slice.716000-716999.json
Success!
1.129 sec

Processing: mpd.slice.717000-717999.json
Success!
1.224 sec

Processing: mpd.slice.718000-718999.json
Success!
1.150 sec

Processing: mpd.slice.719000-719999.json
Success!
1.217 sec

Processing: mpd.slice.72000-72999.json
Success!
1.170 sec

Processing: mpd.slice.720000-720999.json
Success!
1.110 sec

Processi

Success!
1.032 sec

Processing: mpd.slice.829000-829999.json
Success!
1.128 sec

Processing: mpd.slice.83000-83999.json
Success!
0.990 sec

Processing: mpd.slice.830000-830999.json
Success!
1.104 sec

Processing: mpd.slice.831000-831999.json
Success!
1.017 sec

Processing: mpd.slice.832000-832999.json
Success!
1.011 sec

Processing: mpd.slice.833000-833999.json
Success!
0.963 sec

Processing: mpd.slice.834000-834999.json
Success!
1.073 sec

Processing: mpd.slice.835000-835999.json
Success!
0.964 sec

Processing: mpd.slice.836000-836999.json
Success!
1.030 sec

Processing: mpd.slice.837000-837999.json
Success!
0.990 sec

Processing: mpd.slice.838000-838999.json
Success!
1.097 sec

Processing: mpd.slice.839000-839999.json
Success!
1.014 sec

Processing: mpd.slice.84000-84999.json
Success!
1.039 sec

Processing: mpd.slice.840000-840999.json
Success!
1.061 sec

Processing: mpd.slice.841000-841999.json
Success!
1.042 sec

Processing: mpd.slice.842000-842999.json
Success!
1.104 sec

Processi

Success!
1.086 sec

Processing: mpd.slice.950000-950999.json
Success!
1.078 sec

Processing: mpd.slice.951000-951999.json
Success!
1.121 sec

Processing: mpd.slice.952000-952999.json
Success!
1.001 sec

Processing: mpd.slice.953000-953999.json
Success!
1.052 sec

Processing: mpd.slice.954000-954999.json
Success!
1.053 sec

Processing: mpd.slice.955000-955999.json
Success!
1.046 sec

Processing: mpd.slice.956000-956999.json
Success!
0.957 sec

Processing: mpd.slice.957000-957999.json
Success!
0.963 sec

Processing: mpd.slice.958000-958999.json
Success!
0.941 sec

Processing: mpd.slice.959000-959999.json
Success!
1.010 sec

Processing: mpd.slice.96000-96999.json
Success!
1.003 sec

Processing: mpd.slice.960000-960999.json
Success!
0.957 sec

Processing: mpd.slice.961000-961999.json
Success!
0.968 sec

Processing: mpd.slice.962000-962999.json
Success!
0.961 sec

Processing: mpd.slice.963000-963999.json
Success!
0.943 sec

Processing: mpd.slice.964000-964999.json
Success!
1.092 sec

Proces

# Debugging

In [ ]:
time_0 = time()    # Record starting time

# List to record all the tracks that are already in the database
# This prevents adding duplicate entries
track_cache = {}

# Load .json file names from disk
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

# Limit to the first few files
# json_files = json_files[:1]

# Iterate through the selected JSON files
for json_file_name in json_files:
    iter_start = time()
    print('Processing:', json_file_name)
    
    # Load file
    json_file_path = os.path.join(data_folder, json_file_name)
    json_file = open(json_file_path)
    data = json.load(json_file)
    
    # Lists to store pairings
    new_track_pairings = []
    new_pos_list = []
    pid_list = []
    
    # Process the playlists
    for playlist in data["playlists"]:
        # Format the playlist info and obtain its tracks
        playlist, track_list, pos_list = format_playlist(playlist)
        
        # Save formated tracks and positions
        new_track_pairings = new_track_pairings + track_list
        new_pos_list = new_pos_list + pos_list
        pid_list = pid_list + [playlist['pid']]*len(track_list)
    
    # Remove duplicate tracks and update cache
    unseen_tracks, track_cache = find_new_tracks(new_track_pairings, track_cache)
    
    # Create the playlist-track pairings
    new_pairings = create_pairings(new_track_pairings, new_pos_list, pid_list, track_cache)
    
    iter_end = time()
    print('Success!')
    print('%0.3f sec' % (iter_end-iter_start) )
    print()

# Commit changes to the database
# conn.commit()

current_time = time()
print('END')
print('Total time: %0.3f sec' % float(current_time-time_0) )
print('Total time: %0.3f min' % float((current_time-time_0)/60) )

# Continue

In [ ]:
# Commit changes to the database
trans = conn.begin()
try:
    trans.commit()
except:
    trans.rollback()
    raise
finally:
    trans.close()


Verify that we got the correct number of playlist-track pairings.

In [8]:
# Find number of pairings
time_0 = time()    # Record starting time

# Load .json file names from disk
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]
json_files.sort()

# Iterate through the selected JSON files
num_pairings = 0
for json_file_name in json_files:
    # Load file
    json_file_path = os.path.join(data_folder, json_file_name)
    json_file = open(json_file_path)
    data = json.load(json_file)
    
    # Process the playlists
    for playlist in data["playlists"]:
        num_tracks = len(playlist['tracks'])
        num_pairings += num_tracks

current_time = time()
print('END')
print('Total time: %0.3f sec' % float(current_time-time_0) )
print('Total time: %0.3f min' % float((current_time-time_0)/60) )

END
Total time: 162.580 sec
Total time: 2.710 min


In [9]:
print('Number of pairings in jsons:', num_pairings)

Number of pairings in jsons: 66346428


In [7]:
results = conn.execute(db.text("SELECT COUNT(*) FROM pairings"))

N = results.fetchall()[0][0]
print('Number of pairings in SQL:', N)

Number of pairings in SQL: 66346428


## Display subsets of the database
Use these to check that the database was built correctly

In [10]:
pd.read_sql_query('SELECT * FROM tracks INNER JOIN pairings on tracks.tid=pairings.tid WHERE pairings.pid=999', conn)

,tid,track_name,artist_name,album_name,duration_ms,track_uri,artist_uri,album_uri,pid,tid,pos
0,34416,Mind over Matter (Acoustic),Pvris,Acoustic,203649,spotify:track:3rUTBx7gmn1IAAwsrjtnO0,spotify:artist:6oFs3qk4VepIVFdoD4jmsy,spotify:album:33oxbqzo7poSO2IbSMqrPS,999,34416,0
1,6881,You and I,Ingrid Michaelson,Be OK,148400,spotify:track:6zYqyOJuyXDOvWJwKkZ8tg,spotify:artist:2vm8GdHyrJh2O2MfbQFYG0,spotify:album:7o2VLivg95UduHjTMTIEIf,999,6881,1
2,20000,TALK ME DOWN,Troye Sivan,Blue Neighbourhood,237464,spotify:track:0nRuVPZx2iA3KWR4xSzryl,spotify:artist:3WGpXCj9YhhfX11TToZcXP,spotify:album:5ouTDazE4LF9bVJPx1nlgW,999,20000,2
3,34417,Un-Thinkable - I'm Ready,City and Colour,"Covers, Pt. 3",297133,spotify:track:2pbxqEYiXJTvFsybGGgSAi,spotify:artist:74gcBzlQza1bSfob90yRhR,spotify:album:2wCLwVcPgBG1UHszyqV6EA,999,34417,3
4,34418,Addicted To Love,Skylar Grey,Addicted To Love,203600,spotify:track:4v0SA7SC65KJ9ToJm15qCk,spotify:artist:4utLUGcTvOJFr6aqIJtYWV,spotify:album:41MBuPQn9Pha9F7hc4vLLR,999,34418,4
5,1630,Coming Over - filous Remix,James Hersey,Coming Over,198545,spotify:track:2kpziA4mHuM7Na6La4upg1,spotify:artist:0lzV2CiahHRiGd6qpADtPS,spotify:album:24zMrWtj8r3QmczFyWhrPM,999,1630,5
6,34419,I Was Made For Loving You / Please Don't Say Y...,Kina Grannis,I Was Made For Loving You / Please Don't Say Y...,162185,spotify:track:1eBk1yNinU0VgY79rcshYa,spotify:artist:7h4j9YTJJuAHzLCc3KCvYu,spotify:album:2OO0USZLYpbIDXFzaiaJlk,999,34419,6
7,34420,Walk Through Fire,Roxanne Emery,Turn Back,228186,spotify:track:3ELm3eyRhR4tF1ncqzMQEV,spotify:artist:70bIIPbBQeaFzRrIfahMEM,spotify:album:5QQEWF40JI4NuGa4Rqn2GP,999,34420,7
8,34421,Fingertips,Ryan O'Shaughnessy,Fingertips,191572,spotify:track:006AVH7fq061voGXkUiII4,spotify:artist:5xxkw4ElO0WX3v4i1W5kAQ,spotify:album:3MksslTJffj1jqxGkQt8Tm,999,34421,8
9,34422,Tower (Don't Look Down),Skylar Grey,Don't Look Down,240560,spotify:track:2SbF8UR7YvY3ArOH8MqgrC,spotify:artist:4utLUGcTvOJFr6aqIJtYWV,spotify:album:0xpiZPRZ2nzVgINpDHaW4Y,999,34422,9


In [ ]:
pd.read_sql_query('SELECT * FROM playlists', conn).sort_values('pid')

In [ ]:
pd.read_sql_query('SELECT * FROM tracks', conn)

In [ ]:
# This is too big to read in one line
# pd.read_sql_query('SELECT * FROM pairings', conn)

In [11]:
# close the database connection
conn.close()
engine.dispose()